<a href="https://colab.research.google.com/github/Advika-Sharma/Advanced-Java/blob/main/guides/linked/ee-api-colab-setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [78]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [79]:
ee.Authenticate()
ee.Initialize(project='pose-at-sea')  # ✅ Correct

## Test the API

Test the API by printing the elevation of Mount Everest.

## Map visualization

`ee.Image` objects can be displayed to notebook output cells. The following two
examples demonstrate displaying a static image and an interactive map.


### Static image

The `IPython.display` module contains the `Image` function, which can display
the results of a URL representing an image generated from a call to the Earth
Engine `getThumbUrl` function. The following cell will display a thumbnail
of the global elevation model.

### Interactive map

The [geemap](https://github.com/gee-community/geemap)
library can be used to display `ee.Image` objects on an interactive
[ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) map.

The following cell provides an example of using the `geemap.Map` object to
display an elevation model.

## Chart visualization

Some Earth Engine functions produce tabular data that can be plotted by
data visualization packages such as `matplotlib`. The following example
demonstrates the display of tabular data from Earth Engine as a scatter
plot. See [Charting in Colaboratory](https://colab.sandbox.google.com/notebooks/charts.ipynb)
for more information.

In [80]:
roi = ee.Geometry.Rectangle([
    -90.9, 28.4,   # lower-left corner (longitude, latitude)
    -87.9, 29.8    # upper-right corner
])

sentinel1 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(roi)
    .filterDate('2010-04-20', '2010-04-25')
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
)

print("✅ Filtered image count:", sentinel1.size().getInfo())


✅ Filtered image count: 0


In [81]:
# Create median image from filtered collection
median_image = sentinel1.median().clip(roi)


In [82]:
# Define oil spill area points or polygons (label = 1)
oil_points = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([-89.7, 28.8]), {'class': 1}),
    ee.Feature(ee.Geometry.Point([-89.6, 28.85]), {'class': 1}),
    ee.Feature(ee.Geometry.Point([-89.65, 28.9]), {'class': 1})
])

# Define non-oil water area points (label = 0)
non_oil_points = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([-89.1, 29.2]), {'class': 0}),
    ee.Feature(ee.Geometry.Point([-88.95, 29.25]), {'class': 0}),
    ee.Feature(ee.Geometry.Point([-88.8, 29.0]), {'class': 0})
])

# Combine both classes
training_points = oil_points.merge(non_oil_points)


In [83]:
import folium

# Define Deepwater Horizon ROI
roi = ee.Geometry.Rectangle([-90.9, 28.4, -87.9, 29.8])

# Load Sentinel-1 VV band
s1_vv = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2021-01-01', '2021-01-15') \
    .filterBounds(roi) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV')

# Create a median image
median_image = s1_vv.median().clip(roi)

# Compute stats for dynamic visualization
stats = median_image.reduceRegion(
    reducer=ee.Reducer.percentile([2, 98]),
    geometry=roi,
    scale=30,
    maxPixels=1e9
).getInfo()

print("Dynamic VV percentile stats:", stats)

# Use dynamic range for visualization
vis_params = {
    'min': stats['VV_p2'],
    'max': stats['VV_p98'],
    'palette': ['black', 'purple', 'blue', 'cyan', 'white']
}

# Folium helper
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr="Map Data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Patch folium
folium.Map.add_ee_layer = add_ee_layer

# Create folium map
m = folium.Map(location=[29.1, -89.4], zoom_start=7)

# Add image layer
m.add_ee_layer(median_image, vis_params, 'Sentinel-1 VV Dynamic')

# Add bounding box rectangle
folium.Rectangle(
    bounds=[[28.4, -90.9], [29.8, -87.9]],
    color='red',
    fill=False,
    tooltip='Deepwater Horizon ROI'
).add_to(m)

m.add_child(folium.LayerControl())
m



Dynamic VV percentile stats: {'VV_p2': -22.23201172740473, 'VV_p98': -7.764350295593402}


In [84]:
# Sample the image at the training points
training_samples = median_image.sampleRegions(
    collection=training_points,
    properties=['class'],
    scale=10  # Sentinel-1 resolution
)

# Print first few training data samples
print(training_samples.limit(5).getInfo())


{'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['VV']}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '1_0_0', 'properties': {'VV': -19.129556277136857, 'class': 1}}, {'type': 'Feature', 'geometry': None, 'id': '1_1_0', 'properties': {'VV': -20.683728979147915, 'class': 1}}, {'type': 'Feature', 'geometry': None, 'id': '1_2_0', 'properties': {'VV': -19.0187118318232, 'class': 1}}, {'type': 'Feature', 'geometry': None, 'id': '2_0_0', 'properties': {'VV': -18.99039911757579, 'class': 0}}, {'type': 'Feature', 'geometry': None, 'id': '2_1_0', 'properties': {'VV': -17.503918846329277, 'class': 0}}]}


In [85]:
task = ee.batch.Export.table.toDrive(
    collection=training_samples,
    description='oil_spill_training_data',
    fileFormat='CSV'
)
task.start()